In [33]:
# Import required libraries
from gradio_client import Client, handle_file  # Changed from 'file' to 'handle_file'
from pathlib import Path
import time
import json
import shutil
import os

# Initialize client
client = Client("Yifeng-Liu/satellite-image-roofs-auto-annotation")



Loaded as API: https://yifeng-liu-satellite-image-roofs-auto-annotation.hf.space ✔


In [34]:
def process_single_image(image_path, output_dir="output"):
    # Setup output directory
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print("Checking image path...")
    image_exists = Path(image_path).exists()
    print(f"Image exists: {image_exists}")

    if not image_exists:
        print(f"Image not found at path: {image_path}")
        return None

    try:
        # Step 1: Upload image
        print("\nStep 1: Uploading image...")
        img_file = handle_file(image_path)  # Changed from file() to handle_file()
        upload_result = client.predict(
            imge_path=img_file,
            api_name="/upload_image"
        )
        
        # Step 2: Trigger processing
        print("\nStep 2: Triggering processing...")
        process_result = client.predict(
            image_path=img_file,
            files=[],
            slider_value=4,
            api_name="/process_images"
        )
        
        # Extract the paths from process_result
        annotation_path = process_result[1].get('value') if len(process_result) > 1 else None
        annotated_image_path = process_result[2].get('value') if len(process_result) > 2 else None
        
        if annotation_path and annotated_image_path:
            print(f"\nFound result paths:")
            print(f"Annotations: {annotation_path}")
            print(f"Annotated image: {annotated_image_path}")
            
            # Generate output filenames based on input image
            base_name = Path(image_path).stem
            out_annotation = output_path / f"{base_name}_annotations.json"
            out_image = output_path / f"{base_name}_annotated.webp"
            
            # Copy the files
            print("\nSaving results...")
            try:
                # Copy annotation file
                shutil.copy2(annotation_path, out_annotation)
                print(f"Saved annotations to: {out_annotation}")
                
                # Copy annotated image
                shutil.copy2(annotated_image_path, out_image)
                print(f"Saved annotated image to: {out_image}")
                
                return {
                    'annotation_file': str(out_annotation),
                    'annotated_image': str(out_image)
                }
            
            except Exception as e:
                print(f"Error saving files: {str(e)}")
                # If we can't copy, try to read and write the annotation file
                try:
                    with open(annotation_path, 'r') as f:
                        annotations = json.load(f)
                    with open(out_annotation, 'w') as f:
                        json.dump(annotations, f, indent=2)
                    print(f"Saved annotations using direct file read/write")
                    return {'annotation_file': str(out_annotation)}
                except Exception as e2:
                    print(f"Error with direct file handling: {str(e2)}")
                    return None
        else:
            print("No valid result paths found in the process result")
            return None
            
    except Exception as e:
        print(f"Error during processing: {str(e)}")
        return None


In [35]:
img_dir = "../zindi-kuyesera-disaster/predictions-20241228a-test_pre/test_pre/images/"
success_count = 0
fail_count = 0
failed_files = []

for filename in os.listdir(img_dir):
    print(f"\nProcessing file {success_count + fail_count + 1}: {filename}")
    image_path = os.path.join(img_dir, filename)
    results = process_single_image(image_path)

    if results:
        print("\nSuccess!")
        success_count += 1
        for key, path in results.items():
            print(f"{key}: {path}")
    else:
        print("\nProcessing failed")
        fail_count += 1
        failed_files.append(filename)

print("\nProcessing complete!")
print(f"Successfully processed: {success_count} files")
print(f"Failed to process: {fail_count} files")
if failed_files:
    print("\nFailed files:")
    for f in failed_files:
        print(f"- {f}")


Processing file 1: malawi-cyclone_00000204_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/3c078c89f975e69d4303effd5b48a86e9da2551cf37a353e1b443d06de4a6373/bd38085b6a453021_annotations.coco.json
Annotated image: /tmp/gradio/d3fd20b29686e0b4687bd3266ff4ef0223a5f5f4d8b1c9f67a8500c0b51971d5/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000204_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000204_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000204_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000204_pre_disaster_annotated.webp

Processing file 2: malawi-cyclone_00000005_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/240eecf884469e4a02


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/fc14e82e534c3c7b01e3794f4b706328b938abde2dc7da562c4a3a4d00a91c53/607e0bbff2dbd0a6_annotations.coco.json
Annotated image: /tmp/gradio/1be588e2117d54b1ae4140262b4625eaa0899fe1b84114c3b0cbc631f9938f18/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000111_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000111_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000111_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000111_pre_disaster_annotated.webp

Processing file 13: malawi-cyclone_00000017_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/f210eb90278ede24067ed2d9159c5982bed37474448cb4b9c9041f525066ca65/022894321d8d8908_annotations.coco.json
Annotated image: /tmp/gradio/28ce6886e2964ed


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/8c8ecc47edf01f701f98839062660a585bbce54339791c3c1bec0b1ff610a878/3c3dd37708cd9754_annotations.coco.json
Annotated image: /tmp/gradio/e2dd6d9624febd416cf7682c1847c94c2a31ab30a3e2737be73464e450492bff/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000058_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000058_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000058_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000058_pre_disaster_annotated.webp

Processing file 24: malawi-cyclone_00000191_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/b9b95cbbb0c99f0a936d20043d7ab1464eef6610c7cd994711c18169ab7dda07/8d88ea8512c284cd_annotations.coco.json
Annotated image: /tmp/gradio/7064d64b8fb25f8


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/0ef09f452fcacebbe53346e1b39423b7f2b46cda7f091b8393e710704e618d5b/3466638c4c29aaf5_annotations.coco.json
Annotated image: /tmp/gradio/1515fd14aaaa8e3714f9b70e3020ecd24d235e699debd84a2f4154e2d98f4c6b/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000152_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000152_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000152_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000152_pre_disaster_annotated.webp

Processing file 35: malawi-cyclone_00000146_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/82254df22e0893507dad903f0285c54105ebfdbccdd6f4e20763a03174c202a7/92c95e74c5eb71cc_annotations.coco.json
Annotated image: /tmp/gradio/15c9a1de1d21104


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/e4a6c858e2aca05e6f04e30a274cd82abb7ef804583d7e3d99d3a5a9da1e6a2c/071e08bdf8d34478_annotations.coco.json
Annotated image: /tmp/gradio/fed2ecd517246bbc2f37c418a3d591af7b224d826ec4294b27c9f3eefcde5dd4/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000251_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000251_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000251_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000251_pre_disaster_annotated.webp

Processing file 46: malawi-cyclone_00000092_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/f8e858a3fac45af382e10a281bd8fdd6e8d63ad2abd88671c19caff337ac8f51/df41295ded1102ce_annotations.coco.json
Annotated image: /tmp/gradio/67379c9b8afd022

...


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/34d60042aef36c4fa77bae1d24c6c4621b978454d6701f7ea78ff7d9d23b0c9d/517a10f92cb096be_annotations.coco.json
Annotated image: /tmp/gradio/60e95caaadfff1cee88aea272eb3960d4e787e38d5eaddbccf4624e765915c67/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000085_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000085_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000085_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000085_pre_disaster_annotated.webp

Processing file 301: malawi-cyclone_00000022_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/e9bbf2efeb46a774a25fb1112a59eef72e414e7fe41ae61335c6a8a1d02c7221/f02ac2ad5a0c87fe_annotations.coco.json
Annotated image: /tmp/gradio/503a6694b43e45


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/2a61f9e7ea9059d4e39b85b8adb8a13962f49b65ed46f5366512ccaf48583681/7fab72e00c5d5a02_annotations.coco.json
Annotated image: /tmp/gradio/832de4e5397a77e547e5a44251cedaeeb2c8b95f7b8ef17896e8f8976a8dac13/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000141_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000141_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000141_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000141_pre_disaster_annotated.webp

Processing file 312: malawi-cyclone_00000288_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/8426c578fa25ec05db4c320f98dfe79ffa1e291ee902b6049324f8f993fe570a/5270a3277beedf34_annotations.coco.json
Annotated image: /tmp/gradio/552a8b8ef250ec


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/ff5402797b23005a20edb9c63fa2f76fe8685e92846a2e8269f01c0c7bb4ea78/bd3336b60ccf724b_annotations.coco.json
Annotated image: /tmp/gradio/e72155b4ee424f88edbfb9b6029018bcd1ebd9017b8581b60549a165da38f2e8/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000207_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000207_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000207_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000207_pre_disaster_annotated.webp

Processing file 323: malawi-cyclone_00000041_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/d53695118ac11e7fc16070047ca2e918637a792475a16dd6190e11c0d23d358d/ef5cd1a6f5b942ab_annotations.coco.json
Annotated image: /tmp/gradio/d6187d7ae10d1c


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/9063a8655b4ea3af9d78e9417a29e07b666ee2cdb646343eae7482f924196afc/a9583b84b4fb4cb0_annotations.coco.json
Annotated image: /tmp/gradio/c37def7310fa6f035b957836a6e3e702b0c072a4f48296c15030d437b8f2d14c/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000070_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000070_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000070_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000070_pre_disaster_annotated.webp

Processing file 334: malawi-cyclone_00000155_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/494e51ea9fbd2a062ca24fc064dd8462ef2cd853f0029fa84649f9c93b6ba6f9/ca88009275418802_annotations.coco.json
Annotated image: /tmp/gradio/4796d03eb0710f


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/d180c888565692ada3ff268b2566ab12198ca48881cc1c0f24777054fc70b67b/737166f334acc70d_annotations.coco.json
Annotated image: /tmp/gradio/31871dc2571166e8ba5c4c0afab205ce83963dd6c6f637d0e28917145254e489/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000197_pre_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000197_pre_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000197_pre_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000197_pre_disaster_annotated.webp

Processing file 345: malawi-cyclone_00000167_pre_disaster.jpg
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/8374a8b1cab1e3eca6121fae93197719e5d408aa459ae2e634b3418d80b3c26a/48ae19bad06a61d1_annotations.coco.json
Annotated image: /tmp/gradio/fec625b871b7d5

In [36]:
img_dir = "../zindi-kuyesera-disaster/test_images/post_only/"
success_count = 0
fail_count = 0
failed_files = []

for filename in os.listdir(img_dir):
    print(f"\nProcessing file {success_count + fail_count + 1}: {filename}")
    image_path = os.path.join(img_dir, filename)
    results = process_single_image(image_path)

    if results:
        print("\nSuccess!")
        success_count += 1
        for key, path in results.items():
            print(f"{key}: {path}")
    else:
        print("\nProcessing failed")
        fail_count += 1
        failed_files.append(filename)

print("\nProcessing complete!")
print(f"Successfully processed: {success_count} files")
print(f"Failed to process: {fail_count} files")
if failed_files:
    print("\nFailed files:")
    for f in failed_files:
        print(f"- {f}")


Processing file 1: malawi-cyclone_00000340_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/d9f996679950dd2f98b0836eb9f41c84e4128e4c6fc0a1def85cd532dc0290a1/bfe44db1848cbd3a_annotations.coco.json
Annotated image: /tmp/gradio/5511edbd44b8505584dd2f171986c92b8c3a3969e8b9cdbbab67e4617c2da387/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000340_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000340_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000340_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000340_post_disaster_annotated.webp

Processing file 2: malawi-cyclone_00000247_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/8ad4a591013d


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/7278a08c1d35254b27d68e88b200f877a3868695436b41a4641da6d658758d78/eeb05afd4a9a6b40_annotations.coco.json
Annotated image: /tmp/gradio/760cebdc41c2457b11e6c161f906f416f8d1a2968832a2df67afd85a1ce7f938/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000117_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000117_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000117_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000117_post_disaster_annotated.webp

Processing file 13: malawi-cyclone_00000053_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/d999c64bd4f0108b39e95351f9f8e176ed4f71a3e7a5ae5c158727acf5ae9f77/c730c963d967d4e9_annotations.coco.json
Annotated image: /tmp/gradio/01a4865639


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/a55aa3f8e172315a4b1a42ee2f96c7c5dce09943e3e55b3c31d78a9a7a5a9aac/a44e528d7f468f28_annotations.coco.json
Annotated image: /tmp/gradio/e525f6dbecdf2df133fb3b20a018d0f34392a965ad17d28208321ae1bd0c39ec/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000260_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000260_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000260_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000260_post_disaster_annotated.webp

Processing file 25: malawi-cyclone_00000023_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/b757a8fdc92f5d25753b48243ef757a9118e4b996ccfd24f1f0332bf58825a91/ca5d25a74e5abb8c_annotations.coco.json
Annotated image: /tmp/gradio/8796b88240


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/0e8c8b02f1483b294a6dfa0dda703eb5cbcbc2d1ab808c78de89e62fa3a07542/4d385a95dfe41741_annotations.coco.json
Annotated image: /tmp/gradio/a8e7038af4c6741552aad1496777bfd54698da5fb4547046eaff0ed6e8d265d5/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000234_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000234_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000234_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000234_post_disaster_annotated.webp

Processing file 36: malawi-cyclone_00000061_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/290a79476b3862e4b2a785d9bf38c3d7123648acd4416765f2b357e40a324981/86001241fde7d9a1_annotations.coco.json
Annotated image: /tmp/gradio/b1bc050f96


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/63b95397875b48423005728263a4fe63bf5a5dcbeb240c88814e8221ff385662/48fd25e0ea34777c_annotations.coco.json
Annotated image: /tmp/gradio/b81c0cebfdaee14df0e5dde99067490569b40a7d9ca1b11945dc08aaac96466a/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000033_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000033_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000033_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000033_post_disaster_annotated.webp

Processing file 47: malawi-cyclone_00000259_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/446937a3d24c67d5be9f1ec7cc19464e0c76d4f17351b60f40050b6f05aae411/a7228b828b5d5f3f_annotations.coco.json
Annotated image: /tmp/gradio/73c472c0ab

...


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/ea60248dacfbd8714bff659c0dd7dd6b0eeb3bf7a8a444b4fda3494653877855/9a8530937ce409fb_annotations.coco.json
Annotated image: /tmp/gradio/257e06bdc0dbb346215bfd51a5100e4240dee9b27a6765da4095b8210c159ec5/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000093_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000093_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000093_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000093_post_disaster_annotated.webp

Processing file 297: malawi-cyclone_00000092_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/03f1b21a2e09db28c7f13213143409c80aecd53661ea0e7a13ef566dcfe281fb/a456ed2af81690e4_annotations.coco.json
Annotated image: /tmp/gradio/da63bac00


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/044e082514d74efcacf724cd60c151895ef20f1667915659e07ed44da96356d9/93e48d494af52729_annotations.coco.json
Annotated image: /tmp/gradio/632bd1f6d12fdd3fbace3f3afdb388190247cc98a71ba545b9fbf57b4df91398/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000047_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000047_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000047_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000047_post_disaster_annotated.webp

Processing file 308: malawi-cyclone_00000091_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/ccf8b717973aac90ffc91bc4558fd547ab80b7ec1165e2560adb6253eb5e525c/73c23196b86969e5_annotations.coco.json
Annotated image: /tmp/gradio/526640730


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/757b8fd07e2d421ba5acef6eb3d8a53688c1732d6eab7ae6721b15a471a00f0d/e548736e8b60447d_annotations.coco.json
Annotated image: /tmp/gradio/99b5258b640a0ced569a178caf99c1825052ae752a0f1007f29691a8a88e065c/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000336_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000336_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000336_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000336_post_disaster_annotated.webp

Processing file 319: malawi-cyclone_00000339_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/5baee6506e1e3f042553a4fc96cdef3a8af4835c7ba04bcd8aec33126dc44beb/15caf4852599a70e_annotations.coco.json
Annotated image: /tmp/gradio/bad507d37


Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/621820b5a46f90b95f95b19b9e2ab00a7fbcb7067bf3d4e3d0d9e02285c9f88c/ff4b11affab24c89_annotations.coco.json
Annotated image: /tmp/gradio/7bfa49b1201bc93da91ab94a96becb2cf78e6670b49321a2105846375a9ee52c/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000069_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000069_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000069_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000069_post_disaster_annotated.webp

Processing file 330: malawi-cyclone_00000017_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/e87bf97c779df6f774c6ff19e24c1f5ef3cb53d49907218ca5ab3a1fc84ce367/59a809a571c56789_annotations.coco.json
Annotated image: /tmp/gradio/9a3f3920d

Error during processing: timed out

Processing failed

Processing file 341: malawi-cyclone_00000110_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...

Found result paths:
Annotations: /tmp/gradio/330cde7eab163bc3b8e99b2f9046f50dc612c1ac8f07bb6b35e25a9eba2c2353/bd3d7b144a85bd11_annotations.coco.json
Annotated image: /tmp/gradio/b95afecde9cb804627f410c9cf96c9cde97d9ff9e9cf10201c736852ea26ba1c/image.webp

Saving results...
Saved annotations to: output/malawi-cyclone_00000110_post_disaster_annotations.json
Saved annotated image to: output/malawi-cyclone_00000110_post_disaster_annotated.webp

Success!
annotation_file: output/malawi-cyclone_00000110_post_disaster_annotations.json
annotated_image: output/malawi-cyclone_00000110_post_disaster_annotated.webp

Processing file 342: malawi-cyclone_00000160_post_disaster.png
Checking image path...
Image exists: True

Step 1: Uploading image...

Step 2: Triggering processing...
